In [1]:
import os
import psycopg2
import pandas as pd
import re
import shutil
import time
from psycopg2.extras import DictCursor
from contextlib import contextmanager

In [2]:
 # https://www.psycopg.org/docs/extras.html?highlight=dictcursor#psycopg2.extras.DictCursor
 con = psycopg2.connect(database="", user="", password="")
 print("connected with Database successfully")

connected with Database successfully


In [3]:
@contextmanager
def timed_cursor():
    start_time = time.time()
    yield con.cursor(cursor_factory=DictCursor)
    print("--- %s seconds ---" % (time.time() - start_time))

In [4]:
# Create tables
def create_table(sql_file):
    with timed_cursor() as cur:
            query = open(sql_file, 'r') 
            cur.execute(query.read())
            con.commit() # Make the changes to the database persistent

In [5]:
# Execute Script Instead
create_table("../sql/create_table_cve_revs.sql") # Create table cve_revs
create_table("../sql/create_table_cve_revs_js.sql") # Create table cve_revs_js
create_table("../sql/create_table_cve_revs_py.sql") # Create table cve_revs_py

--- 204.39195895195007 seconds ---
--- 642.5687625408173 seconds ---
--- 115.754403591156 seconds ---


In [6]:
def get_cve(row):
    rv = list()
    for regex in regexes:
        rv.extend(list(set(regex.findall(row['msg']))))
    if not rv and regex_sql_inj.search(row['msg']):
        rv.append("sql_injection")
    return rv

In [7]:
def copy_cwe(row):
    if not isinstance(row['cve_data'], str):
        return row['cwe_group']
    if 'CWE' in row['cve_data']:
        return row['cve_data']
    return row['cwe_group']

In [8]:
def create_statistics(df, lang):
    cwe_counts = df['cwe_group'].value_counts()
    cwe_counts.to_csv(os.path.join(res_dir, f'count_cwe_groups_{lang}.csv'), header=False)

    severity_counts = df['severity'].value_counts()
    severity_counts.to_csv(os.path.join(res_dir, f'count_severity_{lang}.csv'), header=False)

    avg_impacts = df['impact_score'].describe()
    avg_impacts.to_csv(os.path.join(res_dir, f'stat_impact_score_{lang}.csv'), header=False)

    stat_yearly = df[['commit_date']]
    stat_yearly = stat_yearly.groupby(stat_yearly.commit_date.dt.year).count()
    stat_yearly.to_csv(os.path.join(res_dir, f'fixes_per_year_{lang}.csv'), header=False)

    cwe_group_year = df[['commit_date', "cwe_group"]]
    cwe_group_year = cwe_group_year.groupby([cwe_group_year.commit_date.dt.year, 'cwe_group']).count()
    cwe_group_year.to_csv(os.path.join(res_dir, f'count_cwe_groups_by_year_{lang}.csv'), header=False)

In [10]:
def select_random_rows(table, limit):
    sql = f"select * from {table} order by random() limit {limit};"
    with timed_cursor() as cur:
        df = pd.read_sql_query(sql, con=con)
        with open(os.path.join(res_dir, f"sample_from_{table}.txt"), 'w') as f:
            for rec_index, rec in df.iterrows():
                f.write(f'------- commit message {rec_index} starts -------\n')
                f.write(rec['msg'])
                f.write('\n------- commit message ends -------\n')

In [ ]:
res_dir = '../../data/results'

regex_cve = re.compile(r"(CVE-\d{4}-\d{4,})", re.IGNORECASE)
regex_cwe = re.compile(r"(CWE-[\d]{1,4})", re.IGNORECASE)
regex_nvd = re.compile(r"(NVD .+)", re.IGNORECASE)
regex_sql_inj = re.compile(r"sql ?injection", re.IGNORECASE)

regexes = [regex_cve, regex_cwe, regex_nvd]

cve_list = pd.read_csv(os.path.join("../../data/cve_parsed", "cve_related_problems.csv"), sep="\t")

if os.path.exists(res_dir):
    shutil.rmtree(res_dir)
os.makedirs(res_dir)
run_on("js")
run_on("py")

# select_random_rows("cve_revs_py", 348)
# select_random_rows("cve_revs_js", 352)